In [ ]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

In [ ]:
from plotly_figures.figure_loader import smartphone_graph_loader

In [ ]:
smartphones = smartphone_graph_loader().update_layout(
        paper_bgcolor="white",
        font_color="black",
        legend_font_color="black",
        title_font_color="white",
        xaxis=dict(color="black"),
        margin=dict(l=30, r=30, b=30, t=30, pad=3),
        showlegend=False
    )

In [ ]:
import plotly.io as pio

pio.write_image(smartphones, "smartphones.png", scale = 2, width=1400, height=800) 

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [ ]:
URL()

In [ ]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [ ]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [ ]:
smartphones_json = to_json(smartphone_list)

In [ ]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [15]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *
from itertools import chain

In [16]:
df_brut = pl.read_json("data/smartphones.json")

### Pipeline

Sur `535` extractions, il y a `487` téléphones valides.

In [17]:
df = NordStream(df_brut)
df

width,screen_type,das_chest,upgrade_storage,ram,model,resolution_1,resolution_2,wifi,battery,das_limbs,price,made_in,cam_1,cam_2,cam_3,height,screen_size,cpu,screen_tech,color,das_head,network,sensor,net_weight,reviews,usb_type_c,brand,fast_charging,thickness,storage,stars,repairability_index,diagonal_pixels,resolution,logprice,induction,ppi,mpx_backward_cam,random_col
f64,str,f64,bool,i64,str,i64,i64,null,i64,f64,f64,str,f64,f64,f64,f64,f64,str,str,str,f64,str,str,f64,i64,bool,str,bool,f64,i64,f64,f64,f64,str,f64,bool,f64,f64,i64
0.7,"""Plat""",1.05,true,8,"""SONY Xperia 5 …",2520,1080,null,5000,2.11,999.0,"""Thaïlande""",12.0,12.0,12.0,15.4,6.1,"""Qualcomm Snapd…","""OLED FHD+""","""Argent""",0.95,"""5G""","""3""",182.0,0,true,"""SONY""",true,0.9,128,0.0,8.5,2741.678318,"""2520 x 1080""",6.906755,true,449.455462,36.0,255
70.6,"""Plat""",0.96,false,8,"""iPhone 15 Pro""",2556,1179,null,3650,2.99,1609.0,"""Chine""",48.0,12.0,12.0,146.6,6.1,"""Puce A17 Pro""","""Super Retina X…","""Bleu""",0.98,"""5G""","""3""",187.0,1,false,"""APPLE""",true,8.2,512,5.0,7.6,2814.813848,"""2556 x 1179""",7.383368,true,461.444893,72.0,184
87.0,"""Plat""",1.59,false,8,"""Samsung Galaxy…",2340,1080,null,3700,2.71,857.59,"""Viêt Nam""",50.0,12.0,10.0,146.0,6.1,"""Exynos 2200""","""Dynamic AMOLED…","""Noir""",1.21,"""5G""","""3""",167.0,1,true,"""SAMSUNG""",true,0.8,256,4.0,8.2,2577.207791,"""2340 x 1080""",6.754126,true,422.493081,72.0,-799
74.7,"""Plat""",1.32,true,4,"""Motorola G53""",1600,720,null,5000,2.98,199.0,"""Chine""",50.0,2.0,0.0,162.7,6.5,"""8 coeurs jusqu…","""LCD""","""Bleu""",0.98,"""5G""","""2""",183.0,3,true,"""MOTOROLA""",false,8.2,128,4.3333,7.1,1754.536976,"""1600 x 720""",5.293305,false,269.928766,52.0,701
7.1,"""Plat""",1.2,false,8,"""Google Pixel 8…",1080,2400,null,4500,2.49,799.0,"""Viêt Nam""",50.0,12.0,10.0,15.1,6.2,"""Google Tensor …","""Actua ou OLED …","""Noir""",0.98,"""5G""","""3""",187.0,0,true,"""GOOGLE""",true,0.9,128,0.0,8.2,2631.805464,"""1080 x 2400""",6.683361,true,424.484752,72.0,668
77.8,"""Plat""",0.98,false,6,"""iPhone 15 Plus…",2796,1290,null,4912,2.98,1499.0,"""Chine""",48.0,12.0,0.0,160.9,6.7,"""Puce A16 Bioni…","""OLED""","""Bleu""",0.98,"""5G""","""2""",201.0,0,true,"""APPLE""",true,7.8,512,0.0,7.5,3079.239517,"""2796 x 1290""",7.312553,true,459.587988,60.0,190
77.1,"""Plat""",1.186,true,2,"""Xiaomi Redmi 9…",1600,720,null,5000,2.526,95.9,"""Chine""",13.0,0.0,0.0,164.9,6.5,"""8 coeurs jusqu…","""HD+""","""Gris""",0.398,"""4G""","""1""",194.0,2,true,"""XIAOMI""",true,9.0,32,4.0,7.0,1754.536976,"""1600 x 720""",4.563306,false,269.928766,13.0,289
73.9,"""Bords incurvés…",1.27,false,8,"""HONOR Magic5 L…",1080,2400,null,5100,2.81,299.0,"""Chine""",64.0,5.0,2.0,161.6,6.7,"""Qualcomm Snapd…","""AMOLED (rigide…","""Noir""",0.82,"""5G""","""3""",175.0,15,true,"""HONOR""",true,7.9,256,4.1333,8.0,2631.805464,"""1080 x 2400""",5.700444,false,392.806786,71.0,399
75.9,"""Plat""",1.4,false,12,"""Google Pixel 6…",3120,1440,null,5000,2.96,633.67,"""Chine""",50.0,48.0,12.0,163.9,6.7,"""Google Tensor""","""Non communiqué…","""Noir""",0.99,"""5G""","""3""",210.0,23,true,"""GOOGLE""",true,8.9,128,4.087,6.4,3436.277055,"""3120 x 1440""",6.451528,true,512.877172,110.0,-839


In [18]:
len(df.columns) - 3

37

df = df.with_columns(
    pl.concat_str([df['resolution_1'], pl.lit(' x '), df['resolution_2']]).alias('Resolution')
)

In [ ]:
from pathlib import Path

root = Path(".").resolve()
data_folder = root / "data"
path_file = data_folder / "df_clean.csv"

In [ ]:
df.write_csv(path_file, separator=";")

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
median = df.select(pl.col("price").median()).item()
mean = df.select(pl.col("price").mean()).item()
std = df.select(pl.col("price").std()).item()
pearson_skewness_coeff = (3*(mean-median))/ std

In [ ]:
pearson_skewness_coeff

Le résultat indique que l'asymétrie de la distribution est positive => indique une distribution décalée à gauche de la médiane

In [ ]:
df.select(cs.by_dtype(pl.NUMERIC_DTYPES)).corr()

# Matrice de corrélation

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.formula as sm
from stargazer.stargazer import Stargazer
import numpy as np
from dataclasses import dataclass
from script.hedonic_regression import *

In [ ]:
y_1 = "price"
y_2 = "logprice"
X = [
    "screen_type",
    "das_limbs",
    "das_head",
    "upgrade_storage",
    "screen_size",
    "fast_charging",
    "repairability_index",
    "made_in",
    "ram",
    "storage",
    "brand",
    "induction",
]

In [ ]:
f_1 = formula(y_1, X)
f_2 = formula(y_2, X)
res_1 = execute_ols(df, f_1)
res_2 = execute_ols(df, f_2)

On ajoute le prix prédit par la régression à notre dataframe

In [ ]:
df = df.with_columns(pl.from_numpy(res_1.predict(), schema=["predicted_price"]))

In [ ]:
Stargazer([res_1, res_2])

In [ ]:
sfa_file = data_folder / "sfa_results.csv"

In [ ]:
sfa_df = pl.read_csv(sfa_file, ignore_errors=True)

In [ ]:
sfa_df.select("model", "price", "predicted_price", "efficiency").sort("efficiency")

In [ ]:
# res2 = res.get_robustcov_results(cov_type='HC1')
# robuste std errors

In [ ]:
import polars as pl

In [ ]:
df = pl.read_json("data/smartphones.json")